In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
# from nltk import word_tokenize

In [2]:
import nltk
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
tweets = pd.read_csv("./Tweets.csv")

In [ ]:
tweets.head()


In [ ]:
tweets.info()

In [ ]:
tweets.shape

In [ ]:
# #this
# review_dataset["tweet_coord"].dropna()
# # or this
# review_dataset["tweet_coord"][~review_dataset["tweet_coord"].isnull()]


In [ ]:
tweets.text.tail()

### Word Cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
negative_tweets = tweets["text"][tweets["airline_sentiment"] =="negative"]
joined_negative_tweets = "".join(negative_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir","JetBlue"])
word_cloud = WordCloud(stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_negative_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




In [ ]:
from wordcloud import WordCloud, STOPWORDS
# Building a word cloud for negative tweet text
# positive_neutral_tweets = tweets["text"][~tweets["airline_sentiment"] =="negative"]
positive_neutral_tweets = tweets["text"][tweets["airline_sentiment"] !="negative"]
joined_positive_neutral_tweets = "".join(positive_neutral_tweets)

my_stop_words = set(STOPWORDS)
# reg = r"@"|
my_stop_words = my_stop_words.union(["@VirginAmerica", "united","flight","AmericanAir","USAirways","SouthwestAir", "JetBlue","VirginAmerica","co", "t"])
word_cloud = WordCloud(background_color="white", stopwords=my_stop_words, regexp=r'\b[^\d\W][^\d\W]+\b').generate(joined_positive_neutral_tweets)

#show word cloud
plt.imshow(word_cloud, interpolation="bilinear")
plt.axis("off")




### Data Preprocessing and Feature Engineering

In [27]:
tweets = tweets[["text","airline_sentiment"]]

In [28]:
#Encode the target label
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
tweets.airline_sentiment = le.fit_transform(tweets.airline_sentiment)

In [29]:
# Defie the text preprocessor function
def preprocessor(text):
    text = re.sub(r"@[d W]+", "", text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', ''))

    return text


In [30]:
# Cleaning the text
tweets.text = tweets.text.apply(preprocessor)

In [10]:
# skip this
# # Add text_length feature
# from nltk import word_tokenize
# tokens = [word_tokenize(word) for word in tweets.text]
# length =[]
# for i in range(len(tokens)):
#     length.append(len(tokens[i]))

# tweets["text_length"] = length

## Building the Classifier

In [31]:
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

#defining the stop words
stop =  set(stopwords.words("english"))
stop_words = stop
stop_words_2 = stop.union({"AmericanAir","USAirways","SouthwestAir", "JetBlue","VirginAmerica"})

def tokenizer(text):
    return [word_tokenize(text)]

def tokenizer_porter(text):
    return [PorterStemmer().stem(text)]

In [32]:
# Define X and y
y = tweets.airline_sentiment
X = tweets.drop('airline_sentiment', axis=1)


In [33]:
from sklearn.model_selection import train_test_split
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=456)

#### Traing a Logistic Regression Model

In [36]:
X_train.shape

(11712, 1)

In [38]:
from  sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = ColumnTransformer([("vectorizer", TfidfVectorizer(lowercase=False),0), ("nothing", "passthrough",[1])] )
tfidf = TfidfVectorizer(lowercase=False)


param_grid =[
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [None, stop_words],
        "vect__tokenizer" : [tokenizer, tokenizer_porter],
        "clf__penalty" : ["l2"],
        "clf__C" : [0.1, 10.0]
    },
    {
        "vect__ngram_range" : [(1,1),(1,2)],
        "vect__stop_words" : [stop_words, stop_words_2],
        "vect__tokenizer" : [tokenizer],
        "vect__use_idf" : [False],
        "vect__norm" : [None],
        "clf__penalty" : ["l2","l1"],
        "clf__solver" : ["saga"],
        "clf__C" : [1.0, 10.0]

    }
]

lr_tfidf = Pipeline([("vect", tfidf), ("clf", LogisticRegression())])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)

gs_lr_tfidf.fit(X_train, y_train)





Fitting 5 folds for each of 32 candidates, totalling 160 fits
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=0.1, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer_porte

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extr

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x79120f97af80>; total time=   0.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x79120f69da20>; total time=   0.0s
[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 2), vect__stop_words=None, vect__tokenizer=<function tokenizer_porter at 0x79120f69da20>; total time=   0.0s
[CV] END

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, clf__solver=saga, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words={'again', 'after', 'same', 'up', 'or', 'its', 'very', 'd', 'further', "you're", 'through', "she's", 'shouldn', 'more', 'all', 'haven', 'his', 'under', 'some', 'needn', 'doing', 'once', 'our', 'are', 'does', 'whom', 'because', 'the', 'while', 'those', 'over', 'did', 'you', 'yours', 'their', 'each', 'no', 'ma', 'were', 'any', "it's", 'will', "should've", 'o', 'why', 's', 'down', 'mightn', "you'd", "mightn't", "you'll", 'this', 'on', "haven't", 'to', 'shan', "hasn't", 'aren', 'other', 'not', 't', 'mustn', 'her', 'and', 'being', 'couldn', 'didn', 'from', 'for', 'nor', 'there', 'into', 'don', 'how', 'have', 'but', 'it', 'itself', 'am', 'she', 'be', 'with', 'hadn', 'do', 'of', 'off', "weren't", 'i', 'out', 'myself', 'isn', 'at', 'if', "aren't", 'ours', "needn't", 'he', 'such', 'themselves', "don't", 'll', 'wouldn', "that'll", 'AmericanAir', 'that', 'had', 'him', 'own', 'has', 

ValueError: 
All the 160 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 469, in fit
    Xt = self._fit(X, y, routed_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 406, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.10/dist-packages/joblib/memory.py", line 

In [26]:
print(gs_lr_tfidf.best_params_)
print(gs_lr_tfidf.best_score_)

{'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__vectorizer__ngram_range': (1, 1), 'vect__vectorizer__stop_words': None, 'vect__vectorizer__tokenizer': <function tokenizer_porter at 0x791211f6ec20>}
0.6420760934418456


In [ ]:
y_predicted = gs_lr_tfidf.best_estimator_.predict(X_test)

In [ ]:
gs_lr_tfidf.best_estimator_.score(X_test, y_test)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

# # Build the vectorizer
# vect = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS, ngram_range=(1, 2), max_features=200,
#                     token_pattern=r'\b[^\d\W][^\d\W]+\b').fit(tweets.text)
# # Create sparse matrix from the vectorizer
# X_tfidf = vect.transform(tweets.text)

# # Create a DataFrame
# X_tfidf_transformed = pd.DataFrame(X_tfidf.toarray(), columns=vect.get_feature_names_out())
# X_tfidf_transformed.head()
# # can also do this: max_df= 0.9, min_df=0.01

In [ ]:
tweets

In [ ]:
data = pd.concat([ X_tfidf_transformed, tweets], axis=1)

In [ ]:
data

In [ ]:
# Lastly, build the classifier


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Train a logistic regression
log_reg = LogisticRegression(solver="newton-cg", C= 1).fit(X_train, y_train)

# Predict the labels
y_predicted = log_reg.predict(X_test)

# Print accuracy score and confusion matrix on test set
print('Accuracy on the test set: ', accuracy_score(y_test, y_predicted))
# print(confusion_matrix(y_test, y_predicted)/len(y_test))
print(confusion_matrix(y_test, y_predicted))